In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader , random_split,SubsetRandomSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm
import pickle
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, precision_score, recall_score, roc_auc_score
import warnings

In [2]:
train_sample = pd.read_csv("/home/wt/meta_target/data/train_dtV2.csv")
gene_feat = pd.read_csv("/home/wt/meta_target/data/all_train_gene_cpg.csv")
exp = pd.read_csv("/home/data/sda/wt/model_data/cell_gene_exp_vs_normal_filter.csv")
sample_info = pd.read_csv("/home/data/sda/wt/model_data/new_model/cell_net_filter_exp/raw/train_cell_info.csv")

In [3]:
class no_gnn(Dataset):
    def __init__(self, sample_info, gene_info, exp):
        self.sample_info = sample_info
        self.gene_info = gene_info
        self.gene_exp = exp
    def __len__(self):
        return len(self.sample_info)

    def __getitem__(self, idx): ###这个idx 相当于原来 train_dtV2 中的行
        gene = self.sample_info.id[idx]
        cell = self.sample_info.cell[idx]
        feat = torch.tensor(self.gene_info.loc[self.gene_info["id"]==gene].drop(columns=["id"]).values.flatten())
        exp = torch.tensor(self.gene_exp.loc[self.gene_exp["cell"]==cell].drop(columns=["cell"]).values.flatten())
        cell_gene_feat = torch.cat((feat,exp))
        ys = self.sample_info.is_dep[idx]
        return cell_gene_feat, ys, gene, cell

In [4]:
train_dt = no_gnn(train_sample, gene_feat, exp)

from sklearn.model_selection import KFold
import random
splits = KFold(n_splits=10,shuffle=True,random_state=2023052701)

In [5]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(11240, 4096),
            torch.nn.ReLU(),
            torch.nn.Linear(4096, 1024),
            torch.nn.ReLU(),
            torch.nn.Linear(1024, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 1)
        )

    def forward(self, x):
        x = self.encoder(x)
        return x

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loss_op = torch.nn.BCEWithLogitsLoss()

In [7]:
from tqdm import tqdm

def train():
    model.train()

    total_loss = 0
    step = 0 
    for _, data in enumerate(tqdm(train_loader)):
        input_x = data[0].to(device)
        ys = data[1].to(device)
        optimizer.zero_grad()
        pred = model(input_x.float())
        loss = loss_op(
            pred,
            ys.float().reshape(-1, 1),
        )
        total_loss += loss.item() 
        loss.backward()
        optimizer.step()
        step = step + 1
    return total_loss / step


@torch.no_grad()
def test():
    model.eval()

    ys, preds, preds_raw, genes = [], [], [], []
    total_loss = 0
    step = 0 
    for _, data in enumerate(tqdm(test_loader)):
        input_x = data[0].to(device)
        label = data[1].to(device)
        ys.append(label.cpu().detach().numpy())
        out = model(input_x.float())
        loss = loss_op(
            out,
            label.float().reshape(-1, 1),
        )
        total_loss += loss.item() 
        step = step + 1
        preds.append(np.rint(torch.sigmoid(out).cpu().detach().numpy()))
        preds_raw.append(torch.sigmoid(out).cpu().detach().numpy())
        gene_name = np.array(data[2])
        genes.append(gene_name)

    all_preds = np.concatenate(preds).ravel()
    all_labels = np.concatenate(ys).ravel()
    all_preds_raw = np.concatenate(preds_raw).ravel()
    all_genes = np.concatenate(genes).ravel()
    
    res = pd.DataFrame({"preds":all_preds,"preds_raw":all_preds_raw,"label":all_labels,"genes":all_genes})
    calculate_metrics(all_preds, all_labels, epoch, "test")
    return total_loss / step, res


def calculate_metrics(y_pred, y_true, epoch, type):
    print(f"\n Confusion matrix: \n {confusion_matrix(y_true, y_pred)}")
    print(f"F1 Score: {f1_score(y_true, y_pred)}")
    print(f"Accuracy: {accuracy_score(y_true, y_pred)}")
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    print(f"Precision: {prec}")
    print(f"Recall: {rec}")
    try:
        roc = roc_auc_score(y_true, y_pred)
        print(f"ROC AUC: {roc}")
    except:
        print(f"ROC AUC: notdefined")

In [8]:
for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(sample_info)))):

    print('Fold {}'.format(fold + 1))
    
    fold_train_idx = [i for i, x in enumerate(train_sample.cell.isin(sample_info.cell[train_idx])) if x] 
    fold_test_idx = [i for i, x in enumerate(train_sample.cell.isin(sample_info.cell[val_idx])) if x] 
    
    train_sampler = SubsetRandomSampler(fold_train_idx)
    test_sampler = SubsetRandomSampler(fold_test_idx)
    train_loader = DataLoader(train_dt, batch_size=1024, sampler=train_sampler, num_workers = 12)
    test_loader = DataLoader(train_dt, batch_size=1024, sampler=test_sampler, num_workers = 12)
    
    model = Net().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    for epoch in range(15):
        loss_train = train()
        print(f"Epoch: {epoch:03d}, Train Loss: {loss_train:.4f}")
        
    loss_test, test_pre = test()
    print(f"Epoch: {epoch:03d}, Test Loss: {loss_test:.4f}")
    test_pre.to_csv("/home/wt/meta_target/data/cv/no_gnn/fold_"+str(fold)+".csv")

Fold 1


100%|██████████| 269/269 [06:35<00:00,  1.47s/it]


Epoch: 000, Train Loss: 0.5874


100%|██████████| 269/269 [06:38<00:00,  1.48s/it]


Epoch: 001, Train Loss: 0.4750


...

Epoch: 013, Train Loss: 0.2778


100%|██████████| 269/269 [06:43<00:00,  1.50s/it]


Epoch: 014, Train Loss: 0.2762


100%|██████████| 31/31 [00:50<00:00,  1.62s/it]



 Confusion matrix: 
 [[21056  2492]
 [ 1825  6362]]
F1 Score: 0.7466697963734522
Accuracy: 0.8639672286119426
Precision: 0.7185452902642874
Recall: 0.7770856235495297
ROC AUC: 0.8356296132016378
Epoch: 014, Test Loss: 0.3281
